<a href="https://colab.research.google.com/github/julycapstone2020/Pneumonia-Detection-Challenge/blob/development/notebook/Pneumonia_Detection_Challenge_SSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End-to-End Pneumonia Detection 

This notebook build an end-to-end build a pneumonia detection system, to locate the position of inflammation in an image.

Tissues with sparse material, such as lungs which are full of air, do not absorb the X-rays and appear black in the image. Dense tissues such as bones absorb X-rays and appear white in the image.

While we are theoretically detecting `lung opacities`, there are lung opacities that are not pneumonia related.

## 1. Problem

To locate the position of inflammation in an image.

## 2. Data

In the data, some of these are labeled `Not Normal No Lung Opacity`.This extra third class indicates that while pneumonia was determined not to be present, there was nonetheless some type of abnormality on the image and oftentimes this finding may mimic the appearance of true pneumonia.

Dicom original images: -Medical images are stored in a special format called DICOM files `(*.dcm)`. They contain a combination of header metadata as well as underlying raw image arrays for pixel data.Details about the data and dataset files are given in below link,

https://www.kaggle.com/c/rsna-pneumonia-detection-challenge/data

### Pre-Processing, Data Visualization, EDA 

* Exploring the given Data files, classes and images of different classes.
* Dealing with missing values
* Visualizationof different classes 
* Analysis from the visualizationof different classes


### Model Building 

* Building a pneumonia detection model starting from basic CNN and then improving upon it.
* Train the model
* To deal with large training time, save the weights so that you can use them when training the model for the second time without starting from scratch


### Test the Model, Fine-tuning and Repeat

* Test the model and report as per evaluation metrics
* Try different models
* Set different hyper parameters, by trying different optimizers, loss functions, epochs, learning rate, batch size, checkpointing, early stopping etc. for these models to fine-tune them
* Report evaluation metrics for these modelsalong with your observation on how changing different hyper parameters leads to change in the final evaluation metric.

## 3. Evaluation



## 4. Features



In [1]:
# Mount google drive

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 1.x
except Exception:
  pass

%tensorflow_version

TensorFlow 1.x selected.
Currently selected TF version: 1.x
Available versions:
* 1.x
* 2.x


In [3]:
# Install 2.1.0 for saving model
!pip install q keras==2.0.4

In [4]:
!git clone https://github.com/pierluigiferrari/ssd_keras

fatal: destination path 'ssd_keras' already exists and is not an empty directory.


In [5]:
!ls 

drive  sample_data  ssd_keras


In [6]:
import os
os.chdir('/content/ssd_keras')

In [7]:
!ls

bounding_box_utils   README.md
CONTRIBUTING.md      ssd300_evaluation_COCO.ipynb
data_generator	     ssd300_evaluation.ipynb
eval_utils	     ssd300_inference.ipynb
examples	     ssd300_pneumonia_training_log.csv
__init__.py	     ssd300_training.ipynb
ISSUE_TEMPLATE.md    ssd512_inference.ipynb
keras_layers	     ssd7_training.ipynb
keras_loss_function  ssd_encoder_decoder
LICENSE.txt	     ssd_keras
misc_utils	     training_summaries
models		     weight_sampling_tutorial.ipynb


In [8]:
from tensorflow.keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, CSVLogger
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms
from eval_utils.average_precision_evaluator import Evaluator

%matplotlib inline

Using TensorFlow backend.


In [9]:
img_height = 300 # Height of the model input images
img_width = 300 # Width of the model input images
img_channels = 3 # Number of color channels of the model input images
mean_color = [123, 117, 104] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
swap_channels = [2, 1, 0] # The color channel order in the original SSD is BGR, so we'll have the model reverse the color channel order of the input images.
n_classes = 2 # Number of positive classes
scales_pascal = [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05] # The anchor box scaling factors used in the original SSD300 for the Pascal VOC datasets
scales_coco = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # The anchor box scaling factors used in the original SSD300 for the MS COCO datasets
scales = scales_pascal
aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] # The anchor box aspect ratios used in the original SSD300; the order matters
two_boxes_for_ar1 = True
steps = [8, 16, 32, 64, 100, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True
model_mode = 'training'

In [10]:
Cur_Directory = '/content/drive/My Drive/Pneumonia Detection/SSD Models'

In [11]:
WeightFile = os.path.join(Cur_Directory,"VGG_ILSVRC_16_layers_fc_reduced.h5")

In [12]:
# 1: Build the Keras model.

#K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, img_channels),
                n_classes=n_classes,
                mode='training',
                l2_regularization=0.0005,
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                two_boxes_for_ar1=two_boxes_for_ar1,
                steps=steps,
                offsets=offsets,
                clip_boxes=clip_boxes,
                variances=variances,
                normalize_coords=normalize_coords,
                subtract_mean=mean_color,
                swap_channels=swap_channels)

# 2: Load some weights into the model.

# TODO: Set the path to the weights you want to load.
weights_path = os.path.join(Cur_Directory,"VGG_ILSVRC_16_layers_fc_reduced.h5")

model.load_weights(weights_path, by_name=True)

# 3: Instantiate an optimizer and the SSD loss function and compile the model.
#    If you want to follow the original Caffe implementation, use the preset SGD
#    optimizer, otherwise I'd recommend the commented-out Adam optimizer.

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
# sgd = SGD(lr=0.001, momentum=0.9, decay=0.0, nesterov=False)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer='adam', loss=ssd_loss.compute_loss)




Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead







Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.


In [13]:
# model.save(os.path.join(Cur_Directory,"model_ssd_initial_1.h5"))

In [14]:
# model_path = os.path.join(Cur_Directory,"model_ssd_initial_1.h5")

In [15]:
# model_trained = os.path.join(Cur_Directory,"ssd300_pneumonia_epoch-39_loss-4.3320_val_loss-4.2565.h5")

In [16]:
# ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

# #K.clear_session() # Clear previous models from memory.

# model = load_model(model_trained, custom_objects={'AnchorBoxes': AnchorBoxes,
#                                                'L2Normalization': L2Normalization,
#                                                'compute_loss': ssd_loss.compute_loss})

In [17]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 300, 300, 3)   0                                            
____________________________________________________________________________________________________
identity_layer (Lambda)          (None, 300, 300, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
input_mean_normalization (Lambda (None, 300, 300, 3)   0           identity_layer[0][0]             
____________________________________________________________________________________________________
input_channel_swap (Lambda)      (None, 300, 300, 3)   0           input_mean_normalization[0][0]   
___________________________________________________________________________________________

In [18]:
!ls 

bounding_box_utils   README.md
CONTRIBUTING.md      ssd300_evaluation_COCO.ipynb
data_generator	     ssd300_evaluation.ipynb
eval_utils	     ssd300_inference.ipynb
examples	     ssd300_pneumonia_training_log.csv
__init__.py	     ssd300_training.ipynb
ISSUE_TEMPLATE.md    ssd512_inference.ipynb
keras_layers	     ssd7_training.ipynb
keras_loss_function  ssd_encoder_decoder
LICENSE.txt	     ssd_keras
misc_utils	     training_summaries
models		     weight_sampling_tutorial.ipynb


In [19]:
# Checkout the labels of our data
import pandas as pd
import numpy as np

PROJECT_DIR = "/content/drive/My Drive/Pneumonia Detection"

# /content/drive/My Drive/Pneumonia Detection/stage_2_train_labels.csv
# /content/drive/My Drive/Pneumonia Detection/stage_2_detailed_class_info.csv

print(os.path.join(PROJECT_DIR , 'stage_2_detailed_class_info.csv'))

class_info_df = pd.read_csv(os.path.join(PROJECT_DIR , 'stage_2_detailed_class_info.csv'))
train_labels_df = pd.read_csv(os.path.join(PROJECT_DIR , 'stage_2_train_labels.csv'))

print(f"Detailed class info -  rows: {class_info_df.shape[0]}, columns: {class_info_df.shape[1]}")
print(f"Train labels -  rows: {train_labels_df.shape[0]}, columns: {train_labels_df.shape[1]}")

/content/drive/My Drive/Pneumonia Detection/stage_2_detailed_class_info.csv
Detailed class info -  rows: 30227, columns: 2
Train labels -  rows: 30227, columns: 6


### Merge both `class info` and `train labels` into one dataset

In [20]:
train_class_df = train_labels_df.merge(class_info_df, left_on='patientId', right_on='patientId', how='inner')

In [21]:
train_class_df.head()

,patientId,x,y,width,height,Target,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0,No Lung Opacity / Not Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0,No Lung Opacity / Not Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0,No Lung Opacity / Not Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1,Lung Opacity


In [22]:
# Fill numeric rows with the zero
for label, content in train_class_df.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
             # Fill missing numeric values with median
            train_class_df[label] = content.fillna(0)

In [23]:
# Check if there are any missing numberic values
train_class_df.isna().sum()

patientId    0
x            0
y            0
width        0
height       0
Target       0
class        0
dtype: int64

In [24]:
train_class_df.columns = ['image_name', 'xmin', 'ymin', 'xmax', 'ymax', 'class_id','class']

In [25]:
train_class_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37629 entries, 0 to 37628
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_name  37629 non-null  object 
 1   xmin        37629 non-null  float64
 2   ymin        37629 non-null  float64
 3   xmax        37629 non-null  float64
 4   ymax        37629 non-null  float64
 5   class_id    37629 non-null  int64  
 6   class       37629 non-null  object 
dtypes: float64(4), int64(1), object(2)
memory usage: 2.3+ MB


In [26]:
train_class_df.drop_duplicates(subset=['image_name', 'xmin', 'ymin', 'xmax', 'ymax' ], keep='first', inplace=True)

train_class_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30227 entries, 0 to 37627
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_name  30227 non-null  object 
 1   xmin        30227 non-null  float64
 2   ymin        30227 non-null  float64
 3   xmax        30227 non-null  float64
 4   ymax        30227 non-null  float64
 5   class_id    30227 non-null  int64  
 6   class       30227 non-null  object 
dtypes: float64(4), int64(1), object(2)
memory usage: 1.8+ MB


In [27]:
 train_class_df = train_class_df[train_class_df['class_id'] == 1]

In [28]:
train_class_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9555 entries, 4 to 37627
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_name  9555 non-null   object 
 1   xmin        9555 non-null   float64
 2   ymin        9555 non-null   float64
 3   xmax        9555 non-null   float64
 4   ymax        9555 non-null   float64
 5   class_id    9555 non-null   int64  
 6   class       9555 non-null   object 
dtypes: float64(4), int64(1), object(2)
memory usage: 597.2+ KB


In [29]:
# Convert float type to int
for label, content in train_class_df.items():
    if pd.api.types.is_float_dtype(content):
      train_class_df[label] = train_class_df[label].astype(int)

In [30]:
train_class_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9555 entries, 4 to 37627
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  9555 non-null   object
 1   xmin        9555 non-null   int64 
 2   ymin        9555 non-null   int64 
 3   xmax        9555 non-null   int64 
 4   ymax        9555 non-null   int64 
 5   class_id    9555 non-null   int64 
 6   class       9555 non-null   object
dtypes: int64(5), object(2)
memory usage: 597.2+ KB


In [31]:
train_class_df = train_class_df[['image_name', 'class_id', 'xmin', 'ymin', 'xmax', 'ymax', 'class']]
train_class_df.head()

,image_name,class_id,xmin,ymin,xmax,ymax,class
4,00436515-870c-4b36-a041-de91049b9ab4,1,264,152,213,379,Lung Opacity
6,00436515-870c-4b36-a041-de91049b9ab4,1,562,152,256,453,Lung Opacity
10,00704310-78a8-4b38-8475-49f4573b2dbb,1,323,577,160,104,Lung Opacity
12,00704310-78a8-4b38-8475-49f4573b2dbb,1,695,575,162,137,Lung Opacity
18,00aecb01-a116-45a2-956c-08d2fa55433f,1,288,322,94,135,Lung Opacity


In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Create pathnames from image Patient ID's
# The directories that contain the images.
Data_Directory = "/content/drive/My Drive/Pneumonia Detection"
images_dir      = os.path.join(Data_Directory,"JPG_Train")

y = train_class_df['class']
X = train_class_df.drop('class', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

print("\nX_train:\n")
print(X_train.head())
print(X_train.shape)

print("\nX_test:\n")
print(X_test.head())
print(X_test.shape)


X_train:

                                 image_name  class_id  xmin  ymin  xmax  ymax
12883  6dc76e37-3793-4ef1-b0ae-e6fa2335e139         1   218   310   235   512
30463  d974dbc0-5e7d-4aac-bda5-f29096aeeb5a         1    82   554   181   205
7569   472dac2e-aba2-4119-b9f7-37939e23e0b1         1   194   461   286   276
37474  3040d9d7-d895-453f-887c-616c10531960         1   641   576   241   183
22542  ac512fbd-8e85-4cc8-9af2-fb035189a72c         1   618   380   203   189
(7644, 6)

X_test:

                                 image_name  class_id  xmin  ymin  xmax  ymax
2718   228b3dc1-f78a-4ac9-b213-2a416583d063         1   284   583   146   143
4952   38c5dfe6-dcca-4bfd-85c1-f3868ab37c6a         1   154   316   205   295
18335  90be7b6e-0db5-4395-843e-d38966f41412         1   102   336   300   614
24283  b42eaeb8-b6d3-41d7-836c-56fd0922abf6         1   590   458   128   228
26540  be27354e-3de7-4420-ae86-05fcc2e9218d         1   160   575   259   243
(1911, 6)


In [33]:
X_train.to_csv(PROJECT_DIR + '/labels_train.csv', index=False, header=False)
X_test.to_csv(PROJECT_DIR + '/labels_val.csv', index=False, header=False)

In [34]:
X_train

,image_name,class_id,xmin,ymin,xmax,ymax
12883,6dc76e37-3793-4ef1-b0ae-e6fa2335e139,1,218,310,235,512
30463,d974dbc0-5e7d-4aac-bda5-f29096aeeb5a,1,82,554,181,205
7569,472dac2e-aba2-4119-b9f7-37939e23e0b1,1,194,461,286,276
37474,3040d9d7-d895-453f-887c-616c10531960,1,641,576,241,183
22542,ac512fbd-8e85-4cc8-9af2-fb035189a72c,1,618,380,203,189
...,...,...,...,...,...,...
15920,819825bd-1bb0-4eef-9299-e2ae28531c2f,1,135,317,271,370
29137,cf8816d8-1032-4b78-8c15-27fbf3b1b763,1,596,431,333,207
5711,3c4ab309-7722-4a52-a340-868b0417a0cc,1,305,498,142,121
5376,3ae1c3de-e2a7-44f5-b794-b1ef71640d0f,1,183,504,223,149


In [35]:


 


# train_images_dir      = os.path.join(Data_Directory,"JPG_Train")
# val_images_dir      = os.path.join(Data_Directory,"JPG_test")
# #VOC_2012_images_dir      = '../../datasets/VOCdevkit/VOC2012/JPEGImages/'
# csv_path_train  = os.path.join(Data_Directory,"stage_2_train_labels.csv")
# csv_path_valid  = os.path.join(Data_Directory,"stage_2_detailed_class_info.csv")

# # The XML parser needs to now what object class names to look for and in which order to map them to integers.
# classes = ['background','No Pneumonia','Pneumonia']

# train_dataset.parse_csv(train_images_dir,labels_filename=csv_path_train,input_format=['image_name','xmin','ymin','xmax','ymax','class_id'],include_classes='all',random_sample=False,ret=False,verbose=True)

# val_dataset.parse_csv(val_images_dir,labels_filename=csv_path_valid,input_format=['image_name','xmin','ymin','xmax','ymax','class_id'],include_classes='all',random_sample=False,ret=False,verbose=True)







# 1: Instantiate two `DataGenerator` objects: One for training, one for validation.

# Optional: If you have enough memory, consider loading the images into memory for the reasons explained above.

train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
val_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)

# 2: Parse the image and label lists for the training and validation datasets.

# TODO: Set the paths to your dataset here.

# Images
# images_dir = '/content/drive/My Drive/Pneumonia Detection/JPG_Train'

Data_Directory = "content/drive/My Drive/Pneumonia Detection"

images_dir      = os.path.join(Data_Directory, "JPG_Train")

print(images_dir)

# Ground truth
train_labels_filename = os.path.join(PROJECT_DIR, 'labels_train.csv')
val_labels_filename   = os.path.join(PROJECT_DIR, 'labels_val.csv')

# The XML parser needs to now what object class names to look for and in which order to map them to integers.
classes = ['background','No Pneumonia','Pneumonia']

# train_dataset.parse_csv(images_dir=images_dir,
#                         labels_filename=train_labels_filename,
#                         input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
#                         include_classes='all')

# val_dataset.parse_csv(images_dir=images_dir,
#                       labels_filename=val_labels_filename,
#                       input_format=['image_name', 'x', 'xmax', 'ymin', 'ymax', 'class_id'],
#                       include_classes='all')

train_dataset.parse_csv(images_dir = images_dir,
                        labels_filename=train_labels_filename,
                        input_format=['image_name','xmin','ymin','xmax','ymax', 'class_id'],
                        include_classes='all',
                        random_sample=False,
                        ret=False,
                        verbose=True)

val_dataset.parse_csv(images_dir = images_dir,
                      labels_filename=val_labels_filename,
                      input_format=['image_name', 'xmin','ymin','xmax','ymax', 'class_id'],
                      include_classes='all',
                      random_sample=False,
                      ret=False,
                      verbose=True)


# Optional: Convert the dataset into an HDF5 dataset. This will require more disk space, but will
# speed up the training. Doing this is not relevant in case you activated the `load_images_into_memory`
# option in the constructor, because in that cas the images are in memory already anyway. If you don't
# want to create HDF5 datasets, comment out the subsequent two function calls.

# train_dataset.create_hdf5_dataset(file_path='dataset_udacity_traffic_train.h5',
#                                   resize=False,
#                                   variable_image_size=True,
#                                   verbose=True)

# val_dataset.create_hdf5_dataset(file_path='dataset_udacity_traffic_val.h5',
#                                 resize=False,
#                                 variable_image_size=True,
#                                 verbose=True)

# # Get the number of samples in the training and validations datasets.
# train_dataset_size = train_dataset.get_dataset_size()
# val_dataset_size   = val_dataset.get_dataset_size()

# print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
# print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))


content/drive/My Drive/Pneumonia Detection/JPG_Train


In [36]:
# 3: Set the batch size.

batch_size = 32 # Change the batch size if you like, or if you run into GPU memory issues.

# 4: Set the image transformations for pre-processing and data augmentation options.

# For the training generator:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.

# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('conv4_3_norm_mbox_conf').output_shape[1:3],
                   model.get_layer('fc7_mbox_conf').output_shape[1:3],
                   model.get_layer('conv6_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv7_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv8_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv9_2_mbox_conf').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.

train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[ssd_data_augmentation],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size)) #7584
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	  5342
Number of images in the validation dataset:	  1765


In [37]:
# Define a learning rate schedule.

def lr_schedule(epoch):
    if epoch < 80:
        return 0.001
    elif epoch < 100:
        return 0.0001
    else:
        return 0.00001

In [38]:
# Define model callbacks.

# TODO: Set the filepath under which you want to save the model.

model_checkpoint = ModelCheckpoint(filepath='ssd300_pneumonia_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)
#model_checkpoint.best = 
csv_logger = CSVLogger(filename='ssd300_pneumonia_training_log.csv',
                       separator=',',
                       append=True)



learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule)

#terminate_on_nan = TerminateOnNaN()

callbacks = [model_checkpoint,
             csv_logger,
             learning_rate_scheduler
             ]
             

In [39]:
# If you're resuming a previous training, set `initial_epoch` and `final_epoch` accordingly.
initial_epoch   = 0
final_epoch     = 1
steps_per_epoch = 50


# history = model.fit_generator(generator=train_generator,
#                               steps_per_epoch=steps_per_epoch,
#                               epochs=final_epoch,
#                               callbacks=callbacks,
#                               validation_data=val_generator,
#                               validation_steps=ceil(val_dataset_size/batch_size),
#                               verbose=1)
                              


history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=final_epoch,
                              callbacks=callbacks,
                              validation_data=val_generator,
                              validation_steps=ceil(val_dataset_size/batch_size),
                              initial_epoch=initial_epoch,
                              verbose=1)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 612, in data_generator_task
    generator_output = next(self._generator)
  File "/content/ssd_keras/data_generator/object_detection_2d_data_generator.py", line 1016, in generate
    with Image.open(filename) as image:
  File "/usr/local/lib/python3.6/dist-packages/PIL/Image.py", line 2809, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: 'content/drive/My Drive/Pneumonia Detection/JPG_Train/ae5bbcb2-8f3a-4f5c-a7b1-1ecaa6a7a967'



Epoch 1/1


ValueError: ignored

In [42]:
!pwd

/content/ssd_keras


In [1]:
!ls -lt /content/drive/My Drive/Pneumonia Detection/JPG_Train


ls: cannot access '/content/drive/My': No such file or directory
ls: cannot access 'Drive/Pneumonia': No such file or directory
ls: cannot access 'Detection/JPG_Train': No such file or directory
